In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import pandas as pd

file_name = 'data/ambient_temperature_system_failure.csv'
df = pd.read_csv(file_name)

def plot_series(df):
    # change the type of timestamp column for plotting
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    # change fahrenheit to °C (temperature mean= 71 -> fahrenheit)
    df['value'] = (df['value'] - 32) * 5/9
    # plot the data
    df.plot(x='timestamp', y='value')
    
plot_series(df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

* Detections that occur too early/late count as misses 